# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [766]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [767]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [768]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [769]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [770]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [771]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [772]:
# Utility lambda method to get k and inertia variables
get_k_inertia = lambda: [list(range(1,11)), []]

In [773]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# Create an empy list to store the inertia values
k, inertia = get_k_inertia()

In [774]:
# Utility method to compute inertia with each possible value of k
def get_inertia_values(k,inertia,df):
    # Create a for loop to compute the inertia with each possible value of k
    for k_value in k:
    # Inside the loop:
    # 1. Create a KMeans model using the loop counter for the n_clusters
        model = KMeans(n_clusters=k_value, random_state=97)
    # 2. Fit the model to the data using `df_market_data_scaled`
        model.fit(df)
    # 3. Append the model.inertia_ to the inertia list
        inertia.append(model.inertia_)

In [775]:
# Utility lambda method to get Inertia values
get_inertia_values(k, inertia,df_market_data_scaled)

In [776]:
# Utility lambda method to get Elbow Curve Data
get_elbow_curve_df = lambda inertia: pd.DataFrame({'k': k, 'inertia':inertia})

In [777]:
# Create a dictionary with the data to plot the Elbow curve
# Create a DataFrame with the data to plot the Elbow curve
elbow_curve_df = get_elbow_curve_df(inertia)

In [808]:
# Method to return elbow curve plot
get_elbow_curve_plot = lambda df, data_type: df.hvplot.line(
    x='k', 
    y='inertia', 
    title=f'Elbow Curve - {data_type} Data', 
    ylabel='Inertia', 
    xlabel='K Value',
    xticks=k,
    width=700,
    height=350,
    grid=True,
    line_width=3,
    xlim=[.9,10.1],
    color='#52B2BF'
   ).opts(
    bgcolor='#ffffff',
    fontsize={
    'title': 18,
    'labels': 15,
    'xticks': 12,
    'yticks': 12})

# Method to return points  
get_scatter = lambda df: df.hvplot.scatter(
    x='k',
    y='inertia',
).opts(color='#E66C2C', size=9, marker='o')

# Method to combine elbow curve with points
get_styled_elbow_curve_plot = lambda df, data_type: get_elbow_curve_plot(elbow_curve_df, data_type) * get_scatter(elbow_curve_df)

In [809]:
# Plot a line chart with all the inertia values computed with the different values of k to visually identify the optimal value for k.
elbow_curve_plot_original = get_styled_elbow_curve_plot(elbow_curve_df,'Original')
elbow_curve_plot_original

:Overlay
   .Curve.I   :Curve   [k]   (inertia)
   .Scatter.I :Scatter   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** The best value for `k` is 4.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [780]:
# Utility method to get and fit KMeans model
def get_fit_model(df):
    model = KMeans(n_clusters=4)
    model.fit(df)
    return model

In [781]:
# Initialize the K-Means model using the best value for k
# Fit the K-Means model using the scaled data
model = get_fit_model(df_market_data_scaled)

In [782]:
# Utility method to get and view predicted clusters
def get_view_predicted_clusters(model,df):
    # Predict clusters
    predicted_clusters = model.predict(df)
    # Print predicted clusters
    print(predicted_clusters)
    # Return predicted cluster
    return predicted_clusters

In [783]:
# Predict the clusters to group the cryptocurrencies using the scaled data
# View the resulting array of cluster values.
predicted_clusters = get_view_predicted_clusters(model,df_market_data_scaled)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [784]:
# Global variable
predicted_cluster = 'Predicted Cluster'

In [785]:
# Utility method to get and view coppied DataFrame
def get_view_copy_df(df,predicted_clusters, is_pca=False):
    df_copy = df.copy()
    df_copy[predicted_cluster] = predicted_clusters
    if is_pca:
        df.rename(columns={0: 'PC1',1: 'PC2',2: 'PC3'})
    display(df_copy.sample(20))
    return df_copy

In [786]:
# Create a copy of the DataFrame
# Add a new column to the DataFrame with the predicted clusters
# Display sample data
df_market_data_scaled_copy = get_view_copy_df(df_market_data_scaled, predicted_clusters)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Predicted Cluster
coin_id,,,,,,,,
dash,-0.296940,0.094763,0.040040,-0.358830,-0.558527,-0.535732,-0.284071,0
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
zcash,-0.127467,0.929119,0.677532,0.223834,-0.437068,-0.265163,-0.214829,2
ethlend,-4.981042,-0.045178,-1.206956,-1.212126,0.047736,4.632380,6.088625,1
omisego,1.919812,0.370447,-1.619761,-0.409716,1.696480,0.811207,-0.021888,0
true-usd,0.078038,-0.687745,-0.009191,-0.058214,0.007388,-0.550441,-0.281747,0
ontology,-0.409044,-0.906963,-1.298986,-1.393153,-0.696937,-0.422835,-0.292344,0
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,2
ethereum-classic,-0.071312,-0.229484,-0.175544,0.051882,-0.551760,-0.526060,-0.273062,0


In [787]:
# Utility method to get and stycle cluster scatter plot
def get_cluster_scatter_plot(df, data_type, x, y): 
    
    # Utility lambda method to format label name
    format_label_name = lambda label_name: label_name.upper().replace('_',' ')
    
    # Returning plot
    return df.hvplot.scatter(
    x=x,
    y=y,
    by=predicted_cluster,
    title=f'Crypto Clusters - {data_type} Data',
    hover_cols=['coin_id'],
    marker=['x','*','^', '+'],
    xlabel=format_label_name(x),
    ylabel=format_label_name(y),
    grid=True,
    width=750,
    height=350,
    line_width=2,
    size=100
).opts(
        bgcolor='#ffffff',
        fontsize={
        'title': 17,
        'legend_title': 13,
        'legend':12,
        'labels': 14,
        'xticks': 12,
        'yticks': 12}
)

In [788]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.
cluster_scatter_plot_original = get_cluster_scatter_plot(df_market_data_scaled_copy, 
                                                         'Original', 
                                                         x="price_change_percentage_24h",
                                                         y="price_change_percentage_7d",)
cluster_scatter_plot_original

:NdOverlay   [Predicted Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [789]:
# Create a PCA model instance and set `n_components=3`.
model = PCA(n_components=3)

In [790]:
# Use the PCA model with `fit_transform` to reduce to three principal components.
pca_model_df = pd.DataFrame(model.fit_transform(df_market_data_scaled))

# View the first five rows of the DataFrame. 
pca_model_df.head()

,0,1,2
0,-0.600667,0.842760,0.461595
1,-0.458261,0.458466,0.952877
2,-0.433070,-0.168126,-0.641752
3,-0.471835,-0.222660,-0.479053
4,-1.157800,2.041209,1.859715


In [791]:
# Retrieve the explained variance to determine how much information can be attributed to each principal component.
model.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 89.5%.

In [792]:
# Global variable
coinid = 'coinid'

In [793]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data
df_market_data_pca = pd.DataFrame(pca_model_df)

# Copy the crypto names from the original data
df_market_data_pca[coinid] = df_market_data.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index(coinid)

# Display sample data
df_market_data_pca.sample(20)

,0,1,2
coinid,,,
crypto-com-chain,-0.248198,-1.376252,-1.462026
maker,-0.389045,0.165041,0.379414
wrapped-bitcoin,-0.604265,0.827398,0.439316
ripple,-0.471835,-0.222660,-0.479053
leo-token,-0.413296,-0.674115,-1.076628
dash,-0.613923,-0.479337,0.339565
havven,0.865762,-2.261882,0.275583
ontology,-0.632355,-2.108117,-0.652227
cardano,-0.345600,0.729439,1.478013


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [794]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
# Create an empy list to store the inertia values
k, inertia = get_k_inertia()

In [795]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
get_inertia_values(k, inertia,df_market_data_pca)

In [796]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve_data = get_elbow_curve_data(inertia)

# Create a DataFrame with the data to plot the Elbow curve
elbow_curve_df = pd.DataFrame(elbow_curve_data)

In [797]:
# Plot a line chart with all the inertia values computed with the different values of k to visually identify the optimal value for k.
elbow_curve_plot_pca = get_styled_elbow_curve_plot(elbow_curve_df,'PCA')
elbow_curve_plot_pca

:Overlay
   .Curve.I   :Curve   [k]   (inertia)
   .Scatter.I :Scatter   [k]   (inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for `k` when using the PCA data is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** This does not differ from the best k value found using the original data.

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [798]:
# Initialize the K-Means model using the best value for k
# Fit the K-Means model using the PCA data
model = get_fit_model(df_market_data_pca)

In [799]:
# Predict the clusters to group the cryptocurrencies using the PCA data
# View the resulting array of cluster values.
predicted_clusters = get_view_predicted_clusters(model,df_market_data_pca)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [800]:
# Create a copy of the DataFrame with the PCA data
# Add a new column to the DataFrame with the predicted clusters
# Display sample data
df_market_data_scaled_copy = get_view_copy_df(df_market_data_scaled, predicted_clusters, True)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Predicted Cluster
coin_id,,,,,,,,
neo,0.286546,-0.326301,-1.212670,-0.903134,0.290970,-0.181187,-0.175550,0
tron,0.130050,-0.041018,0.147155,-0.543776,0.120116,-0.241118,-0.234014,0
digibyte,1.217453,-0.607714,-0.907066,0.449939,-0.662530,0.572367,-0.132482,0
ftx-token,0.414711,0.414044,-0.047386,-0.465380,0.128185,-0.416502,-0.145469,0
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,2
theta-token,-1.612188,-1.682027,-0.816921,1.148607,1.712641,1.502992,0.286977,0
cdai,0.180851,-0.704931,-0.001816,-0.143237,0.016060,-0.551146,-0.282310,0
zcash,-0.127467,0.929119,0.677532,0.223834,-0.437068,-0.265163,-0.214829,2
ontology,-0.409044,-0.906963,-1.298986,-1.393153,-0.696937,-0.422835,-0.292344,0


In [801]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.
cluster_scatter_plot_pca = get_cluster_scatter_plot(df_market_data_pca_copy,'PCA', 'PC1','PC2')
cluster_scatter_plot_pca

:NdOverlay   [Predicted Cluster]
   :Scatter   [PC1]   (PC2)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [802]:
# Composite plot to contrast the Elbow curves
elbow_curve_plot_original + elbow_curve_plot_pca

:Layout
   .Overlay.I  :Overlay
      .Curve.I   :Curve   [k]   (inertia)
      .Scatter.I :Scatter   [k]   (inertia)
   .Overlay.II :Overlay
      .Curve.I   :Curve   [k]   (inertia)
      .Scatter.I :Scatter   [k]   (inertia)

In [803]:
# Compoosite plot to contrast the clusters
cluster_scatter_plot_original + cluster_scatter_plot_pca

:Layout
   .NdOverlay.I  :NdOverlay   [Predicted Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Predicted Cluster]
      :Scatter   [PC1]   (PC2)

#### Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** After visually analyzing the cluster analysis results, the impact of using fewer features to cluster the data using K-Mean results in similar performance to the original as both result in four clusters.